In [401]:
import importlib
import regex as re
import pandas as pd
from tqdm import tqdm
from operator import itemgetter
from unidecode import unidecode

In [457]:
import utils.db as db
import utils.clean_data as c

In [480]:
importlib.reload(c)

<module 'utils.clean_data' from '/Users/andreamarin/Documents/personal_projects/keyword-classifier/data_analysis/utils/clean_data.py'>

In [490]:
conn = db.connect_mongo_db("senate-publication")

ConfigurationError: The resolution lifetime expired after 21.133 seconds: Server Do53:192.168.8.1@53 answered The DNS operation timed out.; Server Do53:192.168.8.1@53 answered The DNS operation timed out.; Server Do53:192.168.8.1@53 answered The DNS operation timed out.; Server Do53:192.168.8.1@53 answered The DNS operation timed out.; Server Do53:192.168.8.1@53 answered The DNS operation timed out.; Server Do53:192.168.8.1@53 answered The DNS operation timed out.; Server Do53:192.168.8.1@53 answered The DNS operation timed out.

In [4]:
conn.list_collection_names()

['initiatives', 'publication', 'propositions', 'unique_publications']

In [5]:
collection = conn.get_collection("unique_publications")

In [6]:
collection_columns = collection.find_one().keys()
collection_columns

dict_keys(['_id', 'type', 'original_summary', 'related_publications', 'involved_authors', 'involved_parties', 'related_dates', 'processed_text', 'topics_distribution'])

# Limpiar datos

## Funciones Generales

In [52]:
select_columns = [
    "_id",
    "original_summary",
    "topics_distribution"
]

In [53]:
def clean_datos_inegi(datos_inegi):
    datos_inegi["estado"] = datos_inegi.estado.apply(str.lower)
    datos_inegi["estado"] = datos_inegi.estado.apply(unidecode)

    datos_inegi["municipio"] = datos_inegi.municipio.apply(str.lower)
    datos_inegi["municipio"] = datos_inegi.municipio.apply(unidecode)

    datos_inegi.loc[datos_inegi.estado == "mexico", "estado"] = "estado de mexico"

    return datos_inegi

In [54]:
datos_inegi_1 = pd.read_csv("./data/catalogo_inegi_clean.csv")
datos_inegi_1 = datos_inegi_1[["NOM_ENT", "NOM_MUN"]].drop_duplicates().rename(
    columns={"NOM_ENT": "estado", "NOM_MUN": "municipio"}
)

datos_inegi_1 = clean_datos_inegi(datos_inegi_1)
datos_inegi_1["exists_inegi_1"] = True

datos_inegi_1.shape

(2473, 3)

In [55]:

datos_inegi_2 = pd.read_csv("./data/catalogo_inegi_seccion_electoral.csv")
datos_inegi_2 = datos_inegi_2[["NOM_ENT", "NOM_MUN"]].drop_duplicates().rename(
    columns={"NOM_ENT": "estado", "NOM_MUN": "municipio"}
)

datos_inegi_2 = clean_datos_inegi(datos_inegi_2)

datos_inegi_2["exists_inegi_2"] = True

datos_inegi_2.shape

(2464, 3)

In [56]:
datos_inegi = datos_inegi_1.merge(
    datos_inegi_2,
    on=["municipio", "estado"],
    how="outer"
)
datos_inegi.shape

(2473, 4)

In [57]:
datos_inegi.isna().sum()

estado            0
municipio         0
exists_inegi_1    0
exists_inegi_2    9
dtype: int64

In [346]:
estados = datos_inegi.estado.unique().tolist()

# add extra estate
estados.append("distrito federal")

# first add the unique named municipios
municipios = datos_inegi.loc[~datos_inegi.municipio.isin(estados)].municipio.unique().tolist()

# add different spellings
municipios.append("praxedis g. de guerrero")
municipios.append("coyame de sotol")

# add municipios that have the same name as a state
municipios.extend(datos_inegi.loc[datos_inegi.municipio.isin(estados)].municipio.unique())

inegi_replacements = {
    "estate": {
        "names_list": estados,
        "base_regex": "[ ,](estados? de )?{name_value}[ ,.]",
        "replacement": " [ESTADO] "
    },
    
    "city": {
        "names_list": municipios,
        "base_regex": "[ ,](municipios? de )?{name_value}[ ,.]",
        "replacement": " [MUNICIPIO] "
    }
}

In [59]:
def explode_topics_data(publications_df: pd.DataFrame) -> pd.DataFrame:
    # explode each model topics into it's own row
    topics_df = publications_df.explode("topics_distribution").reset_index(
        drop = True
    )
    
    # normalize model's topic info into it's own column
    topics_df = topics_df.join(
        pd.json_normalize(topics_df.topics_distribution)
    ).drop(
        columns = ["topics_distribution"]
    )
    
    # explode each topic into it's own row
    topics_df = topics_df.explode("topics").reset_index(drop = True)
    
    # explode each topic's info into columns
    topics_df = topics_df.join(
        pd.json_normalize(topics_df.topics)
    ).drop(
        columns = ["topics", "model_id"]
    )
    
    # sort by topic's score
    topics_df = topics_df.sort_values(by="score", ascending=False).reset_index(drop=True)

    return topics_df

## Limpiar iniciativas

In [19]:
initiatives_cursor = conn.unique_publications.find({"type": "iniciativa"}, select_columns)
raw_initiatives = pd.DataFrame(initiatives_cursor)
raw_initiatives.shape

(5686, 3)

In [20]:
raw_initiatives.head(2)

,_id,original_summary,topics_distribution
0,00024466b3c4102e89e26260242f86f8,Propone establecer que los servicios de salud ...,"[{'model_id': 'normalized_text_12', 'topics': ..."
1,00069181c1c5e63e833a2dc3d6798e9a,Propone establecer que para la obtención de la...,"[{'model_id': 'normalized_text_12', 'topics': ..."


In [21]:
initiatives_df = explode_topics_data(raw_initiatives)
initiatives_df.head(2)

,_id,original_summary,topic,score
0,8e3c8b7a660089d3ab0a6fc9c49d7e5e,Propone establecer que serán sancionados con u...,8,0.98868
1,a32460227a52f000199646c6f7cba2e7,Propone establecer que los comerciantes estén ...,0,0.98839


In [22]:
# keep representative topics
initiatives_df = initiatives_df.loc[initiatives_df.score >= 0.5]

### Exploración por topics

In [23]:
initiatives_df.groupby("topic").agg({
    "_id": "count"
}).reset_index().sort_values(by="_id", ascending=False)

,topic,_id
0,0,663
4,4,446
8,8,351
10,10,259
6,6,229
1,1,204
7,7,136
3,3,108
5,5,69
11,11,14


In [222]:
topic = 2
num_documents = 20

filtered_df = initiatives_df.loc[initiatives_df.topic == topic]

num_row = 0
for _, row in filtered_df.iterrows(): 
    if num_row >= num_documents:
        break
    
    print(f"({row['_id']}) – {round(row['score'], 4)}")
    print("---"*5)
    print(row.original_summary)
    print("======="*10)
    
    num_row += 1

(253fea0c2c8f5eb5ec081d11e76354db) – 0.9765
---------------
Propone que, para los municipios de Coyame del Sotol, Ojinaga y Manuel Benavides, del Estado de Chihuahua, acorde a sus correlativos a su franja fronteriza, se aplico el meridiano 90 grados al oeste de Greenwich y para el Estado de Baja California, así como para los municipios de Janos, Ascensión, Juárez, Praxedis G. Guerrero y Guadalupe, del Estado de Chihuahua, acorde a sus correlativos a su franja fronteriza, se aplica el meridiano 105 grados al oeste de Greenwich.
(0e3953d9b0882adb031d026c9f7bf731) – 0.8472
---------------
Propone que la Ciudad de México sea una entidad federativa libre y soberana.
(c1e483c621cb470de2e3585f1c886752) – 0.6835
---------------
Propone establecer que la Zona Pacífico también comprenderá los municipios de Janos, Ascensión, Juárez, Praxedis G. de Guerrero y Guadalupe del Estado de Chihuahua, lo que implicará que su horario se determine conforme al paralelo 105 en lugar de 90 grados al oeste de G

### Replace common proper names

In [385]:
context_word = "soberanía"

matches = initiatives_df.loc[
    initiatives_df.original_summary.str.contains(context_word)
].original_summary.apply(get_context, context_word=context_word, before_word=True)

set().union(*matches)

{'Propone establecer una soberanía',
 'considerada por esta soberanía',
 'contribución a la soberanía',
 'división implica la soberanía',
 'expeler el amparo soberanía',
 'fortalecimiento de la soberanía',
 'la autonomía y soberanía',
 'la seguridad y soberanía'}

In [484]:
initiative_replacements = [
    {"replacement_regex": "(honorables? )?C[aá]maras? de Senador[ae]s( y Senador[ae]s)?", "replacement": "Camara_Senadores"},
    {"replacement_regex": "(Comisión Permanente del |Cámaras del |Honorable )?Congreso de la Unión", "replacement": "Congreso_Union"},
    {"replacement_regex": "Senado de la República", "replacement": "Senado_Republica"},
    {"replacement_regex": "Secretaría de", "replacement": ""},
     {"replacement_regex": "(esta|una) soberanía", "replacement": "[AUTORIDAD]"},
    {"replacement_regex": "(honorables? )?C[aá]maras? de Diputad[ao]s( y Diputad[ao]s)?", "replacement": "Camara_Diputados"},

]

In [485]:
initiatives_df["clean_summary"] = initiatives_df.original_summary
for replacement_data in initiative_replacements:
    initiatives_df["clean_summary"] = initiatives_df.clean_summary.apply(c.replace_string, **replacement_data)

In [486]:
text = initiatives_df.loc[
    initiatives_df.original_summary.str.contains("honorables Cámara de Senadores")
].clean_summary.iloc[0]
text

'Propone la inscripción de letras de oro en el Muro de Honor del Salón de sesiones de la Camara_Senadores la leyenda “A las sufragistas que nunca se rindieron ni abandonaron la lucha por la libertad”. En dicho reconocimiento se pretende visibilizará la lucha de las mujeres por la conquista de los derechos políticoelectorales.'

### Replace state + city names

In [487]:
clean_state_summaries = []
processing_errors = []

for i, original_text in tqdm(initiatives_df.clean_summary.items(), total=initiatives_df.shape[0]):
    try:
        clean_state_summaries.append(
            c.replace_places_by_flag(original_text, inegi_replacements=inegi_replacements)
        )
    except Exception as ex:
        # save error in list
        clean_state_summaries.append("[ERROR]")
        processing_errors.append((i, original_text, str(ex)))

# save summaries
initiatives_df["clean_state_summary"] = clean_state_summaries

100%|██████████| 2497/2497 [01:08<00:00, 36.43it/s] 


In [488]:
len(processing_errors)

0

In [477]:
initiatives_df.loc[
    (initiatives_df.clean_state_summary.str.contains("\[MUNICIPIO\]"))
    | (initiatives_df.clean_state_summary.str.contains("\[ESTADO\]"))
].shape

(64, 7)

In [472]:
# 103 w/original approach
# 8 w/new approach
replaced_strings = initiatives_df.loc[
    (initiatives_df.clean_state_summary.str.contains("\[MUNICIPIO\]"))
    | (initiatives_df.clean_state_summary.str.contains("\[ESTADO\]"))
]
replaced_strings.shape

(103, 7)

In [474]:
for _, row in replaced_strings.iterrows():
    print(row.original_summary)
    print("---")
    print(row.clean_state_summary)
    print("---")
    print(row.clean_state_summary_v2)
    print("====="*10)

Propone que, para los municipios de Coyame del Sotol, Ojinaga y Manuel Benavides, del Estado de Chihuahua, acorde a sus correlativos a su franja fronteriza, se aplico el meridiano 90 grados al oeste de Greenwich y para el Estado de Baja California, así como para los municipios de Janos, Ascensión, Juárez, Praxedis G. Guerrero y Guadalupe, del Estado de Chihuahua, acorde a sus correlativos a su franja fronteriza, se aplica el meridiano 105 grados al oeste de Greenwich.
---
Propone que, para los [MUNICIPIO]  [MUNICIPIO] y [MUNICIPIO]  del [ESTADO]  acorde a sus correlativos a su franja fronteriza, se aplico el meridiano 90 grados al oeste de Greenwich y para el [ESTADO]  así como para los [MUNICIPIO]  [MUNICIPIO]  [MUNICIPIO]  [MUNICIPIO] y [MUNICIPIO]  del [ESTADO]  acorde a sus correlativos a su franja fronteriza, se aplica el meridiano 105 grados al oeste de Greenwich.
---
Propone que, para los [MUNICIPIO]  [MUNICIPIO] y [MUNICIPIO]  del [ESTADO]  acorde a sus correlativos a su franja

### Update data in mongo

In [478]:
updated_data = initiatives_df[["_id", "clean_summary", "clean_state_summary"]]

In [492]:
updated_data.to_csv("./data/clean_initiatives.csv")

In [ ]:
db.batch_update_publications(
    publications = initiatives_df.to_dict(orient="records"),
    table_name = "unique_publications", 
    conn = conn
)

## Limpiar proposiciones

* (ÚNICO/PRIMERO/.../TERCERO) El Senado de la República, ... exhorta/solicita,? respetuosamente,?
* Gobierno de .*
* (Palacio de )?Gobierno del Estado de .*
* [Ee]stado de .*
* plazo de .* días/meses
* se cumpla con la reforma a los artículos 76 y 78 de la Ley Federal del Trabajo
* [a-z])
* formula respetuoso exhorto

In [ ]:
replacements =[
    "Senado de la República", 
    "Cámara de Senadores", 
    "Honorable Congreso de la Unión", 
    "Comisión Permanente",
    "Congreso de la Unión", 
    "Cámara de Senadores", 
    "Diario Oficial de la Federación", 
    "Soberanía", 
    "esta [sSoberanía]"
]

In [ ]:
proposal_starters = [
    "ÚNICO", 
    "PRIMERO", 
    "SEGUNDO", 
    "TERCERO", 
    "CUARTO",
    "QUINTO"
]

In [ ]:
starter_options = "|".join)

In [ ]:
(ÚNICO/PRIMERO/.../TERCERO) El Senado de la República, ... exhorta/solicita,? respetuosamente,?